In [ ]:
!nvidia-smi

In [ ]:
import os

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import Callback, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

import matplotlib.pyplot as plt

## load the model

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
model = get_unet(3, 512, 512, classes=1)
# tmp = glob.glob('/root/data/')

In [ ]:
# # TEST
# model.load_weights('/root/data/models/erko/segmentation/0904_oneclass_fg_17.h5')
# preds = model.predict(np.expand_dims(np.array(Image.open('/root/data/gopro/frames/553265761/553265761_13518.png').resize((512, 512))), axis=0))

In [ ]:
# plt.figure(figsize=(15, 15))
# plt.imshow(Image.open('/root/data/gopro/frames/553265761/553265761_13518.png').resize((512, 512)))
# plt.imshow(preds[...,1].squeeze(), alpha=0.3)

In [ ]:
lr=0.001
adam = Adam(lr=lr)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

## load data and define generators

In [ ]:
import glob
import cv2
import numpy as np
import random
import math
import json
SEED = 448
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
import shutil
from collections import Counter

In [ ]:
all_images = glob.glob('/root/data/aquabyte-images/erko/raw/*/*.semantic.jpg') + glob.glob('/root/data/aquabyte-images/erko/raw/*/*.semantic.png')

In [ ]:
# images = []
# for (i, mask_path) in enumerate(all_images):
#     if i % 50 == 0:
#         print('{}/{}'.format(i, len(all_images)))
#     mask_img = np.array(Image.open(mask_path)) # .resize((input_shape[0], input_shape[1])))           
#     red, green, blue = mask_img[:,:,0], mask_img[:,:,1], mask_img[:,:,2]
#     if mask_path.endswith('png'):
#         pink_mask = (red == 255) & (green == 105) & (blue == 180)
#     elif mask_path.endswith('jpg'):
#         pink_mask = (red == 255) & (green == 105) & (blue == 179)
#     if np.count_nonzero(pink_mask) > 0:
#         images.append(mask_path)

In [ ]:
# json.dump(images, open('./balanced_images.json', 'w'))

In [ ]:
images = json.load(open('./balanced_images.json'))

In [ ]:
random.seed(SEED)
random.shuffle(images)
cutoff = int(len(images)*0.8)
train = images[:cutoff]
val = images[cutoff:]

In [ ]:
print(len(train))
print(len(val))

In [ ]:
batch_size = 8
steps_per_epoch = len(train) // batch_size
steps_per_epoch_val = len(val) // batch_size
input_shape = (512, 512, 3)
kernel = np.ones((5,5),np.uint8)

In [ ]:
def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x

In [ ]:
def generator(images, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0
    seq = iaa.Sequential([iaa.Sometimes(0.7, iaa.GaussianBlur(sigma=(0, 2.0))),
                  iaa.Sharpen(alpha=(0, 0.1), lightness=(0.7, 1.3)),
                  iaa.ContrastNormalization((0.5, 1.2))],
                 random_order=True)
    img_size = input_shape[0]
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            # img_path = images[j]
            # img_name = os.path.basename(img_path)
            # mask_path = '/root/data/erko/labels/{}.semantic.png'.format(img_name)
            mask_path = images[j]
            img_path = mask_path.replace('.semantic.png', '')
            img_path = img_path.replace('.semantic.jpg', '')
            
            xb = np.array(Image.open(img_path).resize((input_shape[0], input_shape[1])))
                        
            mask_img = np.array(Image.open(mask_path).resize((input_shape[0], input_shape[1])))
            
            mask0 = np.zeros((input_shape[0], input_shape[1]))
            
            red, green, blue = mask_img[:,:,0], mask_img[:,:,1], mask_img[:,:,2]
            if mask_path.endswith('png'):
                pink_mask = (red == 255) & (green == 105) & (blue == 180)
            elif mask_path.endswith('jpg'):
                pink_mask = (red == 255) & (green == 105) & (blue == 179)
            
#            contours = np.logical_and(red>0, red<254) & np.logical_and(green>0, green<254) & np.logical_and(blue>0, blue<=255) 
#            mask0[contours] = 1
            mask0[pink_mask] = 1
            
            y0 = cv2.erode(mask0, kernel)
            
            if np.random.random() > 0.5:
                xb = flip_axis(xb, 1)
                y0 = flip_axis(y0, 1)
                
            x_batch[ind,...] = xb
            y_batch[ind,...,0] = y0
            
        x_batch = seq.augment_images(x_batch)
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
### TEST ####
mask_path = images[1]
kernel = np.ones((7,7),np.uint8)

mask0 = np.zeros((input_shape[0], input_shape[1]))
mask_img = np.array(Image.open(mask_path).resize((input_shape[0], input_shape[1])))
red, green, blue = mask_img[:,:,0], mask_img[:,:,1], mask_img[:,:,2]
# contours = np.logical_and(red>0, red<255) & np.logical_and(green>0, green<=255) & np.logical_and(blue>0, blue<=255) 
if mask_path.endswith('png'):
    pink_mask = (red == 255) & (green == 105) & (blue == 180)
elif mask_path.endswith('jpg'):
    pink_mask = (red == 255) & (green == 105) & (blue == 179)

mask0[pink_mask] = 1
mask0err = cv2.erode(mask0, kernel)

plt.imshow(mask0 - mask0err)
plt.show()
### TEST ####

In [ ]:
train_generator = generator(train, steps_per_epoch, batch_size, input_shape)
val_generator = generator(val, steps_per_epoch_val, batch_size, input_shape)

In [ ]:
X, Y = next(train_generator)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(X[1,...])
# plt.imshow(Y[0,...,0], alpha=0.3)
# plt.imshow(Y[0,...,1], alpha=0.3)
plt.imshow(Y[1,...,0], alpha=0.3)

## train

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = lr
    drop = 0.5
    epochs_drop = 10.0
    fake_epoch = epoch
#     lrate = initial_lrate * math.pow(drop, math.floor((1+fake_epoch)/epochs_drop))
    # if epoch >= 30:
        #fake_epoch = epoch - 20 
#         lrate = initial_lrate * math.pow(drop, math.floor((1+fake_epoch)/epochs_drop))
#     else:
#         lrate = initial_lrate
    lrate = initial_lrate * math.pow(drop, math.floor((1+fake_epoch)/epochs_drop))
    print('lr {}'.format(lrate))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)

In [ ]:
saveh = SaveHistory('./erko_0906_pink_eroded_history.json')

In [ ]:
# save model
filepath = os.path.join('/root/data/models/erko/segmentation/', '0906_pink_eroded_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_jaccard_coef_int', 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=50,
        verbose=1,
        callbacks=[saveh, lr_scheduler, checkpoint],
        validation_data= val_generator,
        validation_steps= steps_per_epoch_val)

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np

In [ ]:
history = json.load(open('./erko_0904_oneclass_fg_history.json'))

In [ ]:
history.keys()

In [ ]:
np.max(history['val_jaccard_coef_int'])

In [ ]:
plt.plot(history['jaccard_coef_int'], label='jaccard_coef_int')
plt.plot(history['val_jaccard_coef_int'], label='val_jaccard_coef_int')
plt.legend()

In [ ]:
plt.plot(history['loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()

# predict

In [ ]:
X, Y = val_generator.next()

In [ ]:
Ypred = model.predict_on_batch(X).squeeze()

In [ ]:
Ypred.shape

In [ ]:
for i in range(8):
    f, ax = plt.subplots(1, 2, figsize=(20, 10))
    
    ax[0].imshow(X[i,...])
    ax[0].set_title("Raw image")
    ax[0].axis('off')
    
    ax[1].imshow(X[i,...])
    ax[1].imshow(Ypred[i,...], alpha=0.3)
    ax[1].axis('off')
    ax[1].set_title("predictions")
    
#     ax[1].imshow(X[i,...])
#     ax[1].imshow(Ypred[i,...,1], alpha=0.3)
#     ax[1].axis('off')
#     ax[1].set_title("foreground")
    
#     ax[2].imshow(X[i,...])
#     ax[2].imshow(Y[i,...,2], alpha=0.5)
#     ax[2].set_title("contours")
#     ax[2].axis('off')
    plt.show()

In [ ]:
plt.imshow(Image.open(val[0]))

# Extract fish

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import copy

In [ ]:
# # img = cv2.imread('coins.png')
# img = Y[-1,...,2]
# test = np.zeros((512, 512, 3))
# test[...,0] = img
# test[...,1] = img
# test[...,2] = img
# img0 = np.zeros_like(img)
# img0[img==0] = 1
# gray = img0
# # img = copy.copy(X[-1,...])
# # img[Y[-1,...,0]==1] = 0
# test = X[-1,...]
# gray = cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [ ]:
# plt.imshow(img)

In [ ]:
import numpy as np
img = np.zeros([512,512,3],dtype=np.uint8)
blue = img[...,2]
blue += Y[0,...,1]*255

In [ ]:
plt.imshow(blue)

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(Y[-1,...,2],cv2.MORPH_OPEN,kernel, iterations = 2)
 
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
 
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
  
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

In [ ]:
plt.imshow(opening)

In [ ]:
plt.imshow(sure_bg)

In [ ]:
plt.imshow(sure_fg)

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
 
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [ ]:
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]

In [ ]:
plt.imshow(markers)

In [ ]:
plt.imshow(img)

# new try

In [ ]:
# import the necessary packages
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import numpy as np
import argparse
import cv2

In [ ]:
img = np.zeros([512,512,3],dtype=np.uint8)
blue = img[...,2]
blue += Y[0,...,1]*255

In [ ]:
# load the image and perform pyramid mean shift filtering
# to aid the thresholding step
# image = cv2.imread(args["image"])
image = img
shifted = cv2.pyrMeanShiftFiltering(image, 21, 51)
# cv2.imshow("Input", image)
plt.imshow(image)

In [ ]:
plt.imshow(shifted)

In [ ]:
# convert the mean shift image to grayscale, then apply
# Otsu's thresholding
gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
# cv2.imshow("Thresh", thresh)
plt.imshow(thresh)


In [ ]:
# compute the exact Euclidean distance from every binary
# pixel to the nearest zero pixel, then find peaks in this
# distance map
D = ndimage.distance_transform_edt(thresh)
localMax = peak_local_max(D, indices=False, min_distance=30, labels=thresh)

In [ ]:
import scipy

In [ ]:
modified_recall = true_positives[-1] / float(num_annotations)
modified_precision = true_positives[-1] / float(true_positives[-1] + false_positives[-1])

def merge_close_markers(markers, thresh=10):
    x, y = np.where(markers!=0)
    dist = scipy.spatial.distance_matrix(zip(x,y), zip(x,y))
    dist = np.triu(dist)
    xclose, yclose = np.where(np.logical_and(dist> 0, dist<=10))
    for (i,j) in zip(xclose, yclose):
        markers[x[i], y[i]] = 0
    return markers

In [ ]:
# perform a connected component analysis on the local peaks,
# using 8-connectivity, then appy the Watershed algorithm
markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
markers = merge_close_markers(markers)
labels = watershed(-D, markers, mask=thresh)
print("[INFO] {} unique segments found".format(len(np.unique(labels)) - 1))

In [ ]:
512 / 4096.0

In [ ]:
plt.imshow(labels)

In [ ]:
for lab in sorted(np.unique(labels))[1:]:
    print lab
    plt.imshow(labels==lab)
    plt.show()
    x, y = np.where(labels==lab)
    bbox = [np.min(x), np.min(y), np.max(x), np.max(y)]
    print(bbox)

In [ ]:
test = labels==3

In [ ]:
plt.imshow(test[int(377*1080/512.0):int(508*1080/512.0), :])

In [ ]:
int(377*1080/512.0), int(508*1080/512.0)

In [ ]:
Image.open(val[0]).size

In [ ]:
plt.imshow(np.array(Image.open(val[0]))[int(377*1080/512.0):int(508*1080/512.0), :])

In [ ]:
np.where(markers!=0)

In [ ]:
import copy

In [ ]:
truc = copy.copy(X[-1,...])

In [ ]:
for label in np.unique(labels): 
    # if the label is zero, we are examining the 'background'
    # so simply ignore it
    if label == 0:
        continue
 
    # otherwise, allocate memory for the label region and draw
    # it on the mask
    mask = np.zeros(gray.shape, dtype="uint8")
    mask[labels == label] = 255
 
    # detect contours in the mask and grab the largest one
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    c = max(cnts, key=cv2.contourArea)
    
    cv2.drawContours(truc, cnts,-1,(255,255,0),3)
    ((x, y), r) = cv2.minEnclosingCircle(c)
    cv2.putText(truc, "#{}".format(label), (int(x) - 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

#     # draw a circle enclosing the object
#     
#     cv2.circle(image, (int(x), int(y)), int(r), (0, 255, 0), 2)
#     
plt.figure(figsize=(15, 15))
plt.imshow(truc)
plt.show()

# Write code to merge labels

In [ ]:
plt.imshow(labels)